In [7]:
import pyuvdata
import numpy as np
import sys
sys.path.append("/opt/devel/rbyrne/rlb_LWA/LWA_data_preprocessing")
import LWA_preprocessing

In [2]:
calfits_file = "/lustre/rbyrne/2024-03-03/20240303_093000-093151_41MHz_reprocess_Dec2024.calfits"
data_filepath = "/lustre/rbyrne/2024-03-03/20240303_093000-093151_41MHz_reprocess_Dec2024.ms"
model_file_name = data_filepath.replace(".ms", "_model.ms")

In [3]:
flag_ants = [
    "LWA009",
    "LWA041",
    "LWA044",
    "LWA052",
    "LWA058",
    "LWA076",
    "LWA095",
    "LWA105",
    "LWA111",
    "LWA120",
    "LWA124",
    "LWA138",
    "LWA150",
    "LWA159",
    "LWA191",
    "LWA204",
    "LWA208",
    "LWA209",
    "LWA232",
    "LWA234",
    "LWA255",
    "LWA267",
    "LWA280",
    "LWA288",
    "LWA292",
    "LWA302",
    "LWA307",
    "LWA309",
    "LWA310",
    "LWA314",
    "LWA325",
    "LWA341",
    "LWA352",
    "LWA364",
    "LWA365",
]

In [4]:
uv = pyuvdata.UVData()
print(f"Reading file {data_filepath}.")
uv.read(data_filepath, data_column="DATA")
uv.set_uvws_from_antenna_positions(update_vis=False)
uv.data_array = np.conj(uv.data_array)
uv.phase_to_time(np.mean(uv.time_array))

Reading file /lustre/rbyrne/2024-03-03/20240303_093000-093151_41MHz_reprocess_Dec2024.ms.


The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 4024.7789143058576 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.
Recalculating uvw_array without adjusting visibility phases -- this can introduce significant errors if used incorrectly.


In [8]:
# Flag outriggers
LWA_preprocessing.flag_outriggers(
    uv,
    inplace=True,
    remove_outriggers=True,
)

In [9]:
# Flag antennas
LWA_preprocessing.flag_antennas(
    uv,
    antenna_names=flag_ants,
    flag_pol="all",  # Options are "all", "X", "Y", "XX", "YY", "XY", or "YX"
    inplace=True,
)

In [10]:
# Calibrate
cal = pyuvdata.UVCal()
cal.read_calfits(calfits_file)
pyuvdata.utils.uvcalibrate(uv, cal, inplace=True, time_check=False)
uv.write_uvfits(data_filepath.replace(".ms", "_calibrated_core.uvfits"))

gain_scale is not set, so there is no way to know what the resulting units are. For now, we assume that `gain_scale` matches whatever is on the UVData object (i.e. we do not change its units). Furthermore, all corrections concerning the pol_convention will be ignored.
pol_convention is not specified on the UVCal object, and uvc_pol_convention was not specified. Tentatively assuming that the UVCal and UVData objects (implicitly) have the same convention.
Neither uvd_pol_convention nor uvc_pol_convention are specified, so the resulting UVData object will have ambiguous convention. 
Times do not match between UVData and UVCal but time_check is False, so calibration will be applied anyway.
UVData object does not have `x_orientation` specified but UVCal does. Matching based on `x` and `y` only 
antnums_to_baseline: found antenna numbers > 255, using 2048 baseline indexing.
Found antenna numbers > 255 in this data set. This is permitted by UVFITS standards, but may cause the `importuvfits` u

In [11]:
model_uv = pyuvdata.UVData()
print(f"Reading file {model_file_name}.")
model_uv.read(model_file_name)
model_uv.phase_to_time(np.mean(uv.time_array))
# Flag outriggers
LWA_preprocessing.flag_outriggers(
    model_uv,
    inplace=True,
    remove_outriggers=True,
)
model_uv.write_uvfits(model_file_name.replace(".ms", "_core.uvfits"))

Reading file /lustre/rbyrne/2024-03-03/20240303_093000-093151_41MHz_reprocess_Dec2024_model.ms.


The uvw_array does not match the expected values given the antenna positions. The largest discrepancy is 1.9095988568870794 meters. This is a fairly common situation but might indicate an error in the antenna positions, the uvws or the phasing.
The entry name zenith_at_jd2460372.896482 is not unique inside the phase center catalog, adding anyways.
The provided name zenith_at_jd2460372.896482 is already used but has different parameters. Adding another entry with the same name but a different ID and parameters.
antnums_to_baseline: found antenna numbers > 255, using 2048 baseline indexing.
Found antenna numbers > 255 in this data set. This is permitted by UVFITS standards, but may cause the `importuvfits` utility within CASA to crash. If attempting to use this data set in CASA, consider using the measurement set writer method (`write_ms`) instead.


In [31]:
uv.filename = [""]
model_uv.filename = [""]
uv.sum_vis(model_uv, difference=True, inplace=True, override_params=[
    "scan_number_array", "phase_center_id_array", "telescope", "phase_center_catalog", "filename", "phase_center_app_dec", "nsample_array", "integration_time", "phase_center_frame_pa", "flag_array", "uvw_array", "lst_array", "phase_center_app_ra"
])

Keyword observer in _extra_keywords is different in the two objects. Taking the first object's entry.


In [32]:
uv.write_uvfits(data_filepath.replace(".ms", "_calibrated_res_core.uvfits"))

antnums_to_baseline: found antenna numbers > 255, using 2048 baseline indexing.
Found antenna numbers > 255 in this data set. This is permitted by UVFITS standards, but may cause the `importuvfits` utility within CASA to crash. If attempting to use this data set in CASA, consider using the measurement set writer method (`write_ms`) instead.
